In [1]:
# pip install bertopic[spacy]

# installés en même temps :
# SentenceTransformer
# plotly
# (plotly pour visualizations de BERTopic)

In [2]:
import pandas as pd
# surveillance des GPU/RAm/etc.
import GPUtil
# vectorisation des textes
import torch
from sklearn.feature_extraction.text import CountVectorizer

# modèle
from bertopic import BERTopic
# error ou warning : c:\Users\ehess\anaconda3\envs\my4env\lib\site-packages\umap\distances.py:1063: 
# NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. 
# The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. 
# See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.@numba.jit()

c:\Users\ehess\anaconda3\envs\my4env\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\ehess\anaconda3\envs\my4env\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\ehess\anaconda3\envs\my4env\lib\site-packages\umap\distances.py:1086: NumbaDepr

In [3]:
import spacy

In [4]:
torch.cuda.is_available()


True

In [5]:
# pour un ordi avec GPU
is_using_gpu = spacy.prefer_gpu()
if is_using_gpu:
    print("Using GPU!")
    torch.set_default_tensor_type("torch.cuda.FloatTensor")
    print("GPU Usage")
    GPUtil.showUtilization()

Using GPU!
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  8% | 48% |


In [6]:
df = pd.read_csv("df_CC_lemmas_postag_corrected.csv", sep = ",", encoding = "utf-8", dtype= str)

# 20 sc

In [7]:
# df = df.sample(10000)

In [9]:
df["lemmas_without_stopwords"] = df["lemmas without stopwords"]
# la fonction .tolist() n'accepte pas les espace dans les noms de colonnes

In [10]:
lemmatized = df.lemmas_without_stopwords.tolist()
lemmatized[30]

"['augmenter', 'salaire', 'fonctionnaire', 'diminuer', 'taxe', 'service', 'revoir', 'convention', 'aide', 'domicile', 'prix', 'kilomètre', 'plupart', 'travailler', 'voiture', 'personnel', 'prix', 'proposer', 'prendre', 'charge', 'réalité', 'voiture', 'maire', 'm', 'besoin', 'entretien', 'pourtant', 'travail', 'domicile', 'permettre', 'accompagner', 'difficulté', 'social', 'malade', 'cadre', 'papier', 'administ', 'extérieure', 'prix', 'association', 'institution', 'temps', 'réfléchir', 'lire', 'comprendre', 'développement', 'ligne', 'ferroviaire', 'olorov', 'augmentation', 'horaire', 'cadencement', 'buzy', 'solution', 'permettre', 'diminuer', 'nombre', 'voiture', 'route', 'accidenter', 'gérer', 'préserver', 'environnement']"

In [11]:
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 | 13% | 49% |


In [12]:
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
# from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer


In [14]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [15]:
print(device)

cuda:0


In [16]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2", device = "cuda") # il le trouve

In [17]:
# Step 1 - Extract embeddings

# the trained embedding produced with the La Javaness procedure, from CamemBert didn t work :
# embedding_model = SentenceTransformer(os.path.join('/Users/ehess/.cache/torch/sentence_transformers/dangvantuan_sentence-camembert-large/', 'sentence_bert_config.json'))
# '/Users/ehess/.cache/torch/sentence_transformers/dangvantuan_sentence-camembert-large'

# pour telecharger :
# from transformers.pipelines import pipeline
# embedding_model = pipeline("feature-extraction", model="dangvantuan/sentence-camembert-large")


In [18]:

# Step 1 - Extract embeddings


embeddings = embedding_model.encode(lemmatized, show_progress_bar=True)
# 56mn 8.6sc quand même


Batches:   0%|          | 0/7033 [00:00<?, ?it/s]

In [19]:
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% | 59% |


In [20]:
# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')


In [21]:

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)


In [22]:
# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer()


In [23]:

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()


In [24]:
# Step 6 - (Optional) Fine-tune topic representations with a `bertopic.representation` model

# representation_model = KeyBERTInspired()

In [25]:
# initialisation de BERTopic. On choisit de suivre les conseils de Maartenger pour réduire la consommation de mémoire :  set low_memory to True, set calculate_probabilities to False
# All steps together
topic_model = BERTopic(
    low_memory = True, 
    calculate_probabilities = False,
    embedding_model=embedding_model,          # Step 1 - Extract embeddings
    umap_model=umap_model,                    # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,              # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
    verbose = True
)
# representation_model=representation_model # Step 6 - (Optional) Fine-tune topic represenations
# topic_model = topic_model.to(device)
# à suivre, il est aussi possible de "set the minimum frequency of words in the CountVectorizer class to reduce the size of the resulting sparse c-TF-IDF matrix." avec la commande suivante :
# vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english", min_df=10)
# The min_df parameter is used to indicate the minimum frequency of words. Setting this value larger than 1 can significantly reduce memory.

In [28]:
topics, probs = topic_model.fit_transform(lemmatized)

Batches:   0%|          | 0/7033 [00:00<?, ?it/s]

2023-05-11 19:17:20,961 - BERTopic - Transformed documents to Embeddings
2023-05-11 19:21:49,759 - BERTopic - Reduced dimensionality
2023-05-11 19:22:22,443 - BERTopic - Clustered reduced embeddings


In [35]:
topic_model.save("BERTopic_CC_model")

c:\Users\ehess\anaconda3\envs\my4env\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [36]:
# nombre de docs associés à un topics (il peut y avoir un doc avec plusieurs topics)
topic_model.get_topic_freq().head(30)

,Topic,Count
0,-1,153970
1,0,3677
2,1,3630
3,2,3238
4,3,2652
5,4,2181
6,5,2148
7,6,1869
8,7,1524
9,8,1259


In [37]:
fig = topic_model.visualize_barchart(top_n_topics=30)
fig.show()

In [39]:
# voir le détail des mots associés à un topic (defini par son num)
topic_model.get_topic(0)

[('commune', 0.008411323995234267),
 ('communauté', 0.00392758548546405),
 ('rural', 0.003717006240757502),
 ('habitant', 0.003274138798294203),
 ('communal', 0.002765976424006859),
 ('compétence', 0.0026164638492775687),
 ('département', 0.0025945814772607625),
 ('village', 0.002435490778005829),
 ('région', 0.0024035677482141782),
 ('maire', 0.0022275627761824904)]

In [31]:
topic_model.get_topic(2)

[('énergie', 0.00466758304589778),
 ('agriculture', 0.004028898291034726),
 ('pesticide', 0.0038518687989596086),
 ('nucléaire', 0.0037638732036206395),
 ('production', 0.0036626400048839543),
 ('électrique', 0.003484209140731056),
 ('bio', 0.0032755953530161468),
 ('pollution', 0.003268460336512557),
 ('écologique', 0.003065889205706307),
 ('eau', 0.003000250904958597)]

In [42]:
# here i am reducing embeddings and creating a list of topics id for lowering the time of the map computation
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
first_100_topics = list(range(100))

In [44]:
figure_topic = topic_model.visualize_documents(lemmatized, topics = first_100_topics, reduced_embeddings=reduced_embeddings)

# Two options :

# Run the visualization with the original embeddings
# topic_model.visualize_documents(lemmatized, embeddings=embeddings)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
# reduced_embeddings = reducer(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# topic_model.visualize_documents(lemmatized, embeddings)

In [46]:
figure_topic.write_html("figure_topic_docs.html")

In [ ]:
# montre tous les topics et leurs mots associés. si default, la liste peut être longue.
topic_model.get_topics()

In [ ]:
#  les echantillons sont trop petits pour avoir des topics visualisés
fig = topic_model.visualize_topics()
fig.show()

In [ ]:
topic_model.visualize_documents(lemmatized)

In [ ]:
topic_model.visualize_hierarchy()